In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
GPU_ID= '0'
os.environ["CUDA_VISIBLE_DEVICES"] =GPU_ID
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

In [ ]:
import keras
# ^^^ pyforest auto-imports - don't write above this line
# example of loading a pix2pix model and using it for image to image translation
from keras.models import load_model
from numpy import load
from numpy import vstack
from matplotlib import pyplot
from numpy.random import randint
import numpy as np
from PIL import Image

In [ ]:
data_dir = "SEN12MS/processed"
model_fn = "model_088000.h5"
testing_states=['ROIs1158_spring_s2_test_patches']

In [ ]:
test_patches=[]
for state in testing_states:
    fn= os.path.join(data_dir,'%s/' %state)
    with os.scandir(fn) as entries:
        for entry in entries:
            #print(entry.name)
            with os.scandir(fn+entry.name) as folders:
                for folder in folders:
                    #print(folder.name)
                    with os.scandir(fn+entry.name+'/'+folder.name) as files:
                        for file in files:
                            #print(file.name)
                            test_patches.append((os.path.join(fn+entry.name+'/'+folder.name+'/',file.name),state))

In [ ]:
#Visualizing the image
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return ((array - array_min) / (array_max - array_min))

In [ ]:
# load and prepare training images
def load_real_samples(training_patches):
    # load compressed arrays
    X1= np.zeros((1,256,256,8))
    X2= np.zeros((1,256,256,8))
    #print(training_patches[0])
    filename= training_patches[0]
    #print(filename)
    data = np.load(filename)
    # unpack arrays
    """redn = normalize(data[:, :, 0])
    greenn = normalize(data[:, :, 1])
    bluen = normalize(data[:, :, 2])
    infraredn = normalize(data[:, :, 3])
    X1[0] = np.dstack((redn, greenn, bluen,infraredn))"""
    #X2[0] = np.copy(np.expand_dims(data[:, :, 5],axis=2))
    #################################for low res########################################33
    """y_train_nlcd = np.copy(data[:, :, 4])
    y_train_nlcd[y_train_nlcd == 1] = 0# forest
    y_train_nlcd[y_train_nlcd == 2] = 0
    y_train_nlcd[y_train_nlcd == 3] = 0
    y_train_nlcd[y_train_nlcd == 4] = 0
    y_train_nlcd[y_train_nlcd == 5] = 0
    y_train_nlcd[y_train_nlcd == 6] = 1  # shrublands
    y_train_nlcd[y_train_nlcd == 7] = 1
    y_train_nlcd[y_train_nlcd == 8] = np.random.choice(np.arange(0, 8), p=[0.15, 0.05, 0.15, 0, 0, 0.15, 0.5, 0])  # savannas
    y_train_nlcd[y_train_nlcd == 9] = np.random.choice(np.arange(0, 8), p=[0.15, 0.05, 0.15, 0, 0, 0.15, 0.5, 0])
    y_train_nlcd[y_train_nlcd == 10] = 2  # grassland
    y_train_nlcd[y_train_nlcd == 11] = 3  # wetlands
    y_train_nlcd[y_train_nlcd == 12] = 4  # croplands
    y_train_nlcd[y_train_nlcd == 14] = 4  # cropland
    y_train_nlcd[y_train_nlcd == 13] = 5  # builtup
    y_train_nlcd[y_train_nlcd == 15] = 7  # ice
    y_train_nlcd[y_train_nlcd == 17] = 7  # water
    y_train_nlcd[y_train_nlcd == 16] = 6  # barren
    #y_train_nlcd = keras.utils.to_categorical(y_train_nlcd, 8)
    y_train_nlcd = keras.utils.to_categorical(y_train_nlcd, 8)"""
    ####################only for high res####################3
    y_train = np.copy(data[:, :, 5])
    y_train[y_train == 1] = 0
    y_train[y_train == 2] = 1
    y_train[y_train == 4] = 2
    y_train[y_train == 5] = 3
    y_train[y_train == 6] = 4
    y_train[y_train == 7] = 5
    y_train[y_train == 9] = 6
    y_train[y_train == 10] = 7
    y_train = keras.utils.to_categorical(y_train, 8)
    ################till here#######################33
    X1[0] =y_train_nlcd
    X2[0]=None
    #print(y_train_nlcd.shape)
    return [X1, X2]

In [ ]:
model = load_model(model_fn)

In [ ]:
lab2rgb = {0:[0./255, 128./255, 0./255],        #dark-green         ->          Forest
           1:[205./255, 133./255, 63./255],     #peruvian-brown     ->          Dense-Shrublands
           2:[224./255, 216./255, 202./255],    #savana-oaks        ->          Woody-Savanas
           3:[102./255, 102./255, 25./255],    #dark-olive         ->          Grasslands
           4:[210./255, 209./255, 205./255],   #concrete           ->          Urban
           #5:[229./255, 229./255, 178./255],   #pale-olive         ->          Vegetation
           5:[255./255, 255./255, 255./255],   #white              ->          Snow-and-ice
           6:[131./255, 117./255, 96./255],    #barren-paint       ->          Barren
           7:[135./255, 206./255, 250./255],   #light-blue         ->          Water-bodies
          }
rgb = np.ones((256,256,3))

In [ ]:
# plot source, generated and target images
def plot_images(src_img, gen_img, tar_img):
    #images = vstack((src_img, gen_img, tar_img))
    # scale from [-1,1] to [0,1]
    #images = (images + 1) / 2.0
    #titles = ['Source', 'Generated', 'Expected']
    # plot images row by row
    
    n_samples =1
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(src_img[i])
    # plot generated target image
    for i in range(n_samples):
        label = np.argmax(gen_img[i], axis=-1)
        for p in range(256):
            for q in range(256):
                rgb[p,q,:] = np.expand_dims(np.array(lab2rgb[label[p,q]]),axis=0)
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(rgb)
    # plot real target image
    for i in range(n_samples):
        label = np.argmax(tar_img[i], axis=-1)
        for p in range(256):
            for q in range(256):
                rgb[p,q,:] = np.expand_dims(np.array(lab2rgb[label[p,q]]),axis=0)
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(rgb)

In [ ]:
for i,namee in enumerate(test_patches):
    [src_image, ___] = load_real_samples(test_patches[i])
    #print(src_image.shape)
    gen_image = model.predict(src_image)
    #plot_images(src_image, gen_image, tar_image)
    outpp=np.argmax(gen_image[0], axis=-1)
    #print(outpp.shape)
    outpp[outpp == 7] = 10
    outpp[outpp == 6] = 9
    outpp[outpp == 5] = 7
    outpp[outpp == 4] = 6
    outpp[outpp == 3] = 5
    outpp[outpp== 2] = 4
    outpp[outpp == 1] = 2
    outpp[outpp== 0] = 1
    #plot_images(src_image, gen_image, tar_image)
    #print(namee[0].replace("ROIs1158_spring_s2_test_patches/ROIs1158_spring/s2_2","dfc_1").replace("s2","dfc").replace(".npy",".tif"))
    st = namee[0].replace("ROIs1158_spring_s2_test_patches/ROIs1158_spring/s2_2","dfc_1").replace("s2","dfc").replace(".npy",".tif")
    dataa = outpp.astype(np.uint8)
    im = Image.fromarray(dataa)
    im.save(st)